In [1]:
import os
import sys
parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    
from src.base_functions import turn_voltage
from src.superconductor_losses import thermal_incomes, cryo_surface,cryostat_losses

The following code snippets try to check the correctness of the transformer parameters, defined by doi.org/10.1088/0953-2048/20/5/010

The turn voltage calculated in 2 different ways:
   - from the core parameters
   - from the given line/phase voltages and turn number of the windings

The given parameters of the transformer core are:
 - the core diameter is 396 mm, the core radius is 198 mm
 - the flux density in the core is 1.275 T
 - the filling factor of the core was not given, we calculating by a usual 88 %
 - the network frequency is 50 Hz
 
Using the turn voltage base function of the library:

In [2]:
ut = turn_voltage(1.275, 198.0, 0.67, 50.0)
print('turn voltage:',round(ut,2), 'V')

turn voltage: 23.36 V


Turn voltage can be calculated from the winding parameters of the star/star connected three phase transformer:

In [3]:
# seconday - outer, primary inner winding
Np = 262.0 # turns in the primary winding
Up = 10500 # V 
utp = Up/Np/1.73
print("Turn voltage from primary winding data: ", round(utp,2), "V")

# secondary winding
Ns = 10
Us = 400
uts= Us/1.73/Ns
print("Turn voltage from secondary winding data:",round(uts,2), "V")

# BSSCO tape dimensions
w = 4.8  # mm
h = 0.32 # mm
A = 4.8*0.32 # 1.536 mm2

# Winding areas
a_lv = 8. * 342.5 # mm
a_hv = 13.5* 355

Nd = 23 # number of discs/pancakes
ff_lv = A*Np/a_lv*100
ff_hv = A*Ns*Nd/a_hv*100

print('Filling factor in LV (primary) winding', round(ff_lv,2), '%')
print('Filling factor in HV (secondary) winding',round(ff_hv,2), '%')

I1 = 34.5
I2 = 902.

ae_lv = (a_lv*(ff_lv/100))
ae_hv = (a_hv*(ff_hv/100))

print("Amperturns should balanced:", I1*Np, I2*Ns)

j_lv = I1*Np/ae_lv
j_hv = I2*Ns/ae_hv

print('j in LV:',round(j_lv,2), 'A/mm2', round(ae_lv,1), round(a_lv,1))
print('j in HV:',round(j_hv,2), 'A/mm2', round(ae_hv,1), round(a_hv,1))

Turn voltage from primary winding data:  23.17 V
Turn voltage from secondary winding data: 23.12 V
Filling factor in LV (primary) winding 14.69 %
Filling factor in HV (secondary) winding 7.37 %
Amperturns should balanced: 9039.0 9020.0
j in LV: 22.46 A/mm2 402.4 2740.0
j in HV: 25.53 A/mm2 353.3 4792.5


Therefore, some parameter of the core is wrong, maybe the flux density of the core is a bit lower in the practice, in this case the flux density should be 0.963 T

In [4]:
from importlib_resources import files
from src.two_winding_model import TransformerDesign, TwoWindingModel

path = files("data").joinpath("630kVA_sc_transformer.json")

import json

with open(path) as json_file:
    data = json.load(json_file)

transformer = TransformerDesign.from_dict(data)

trafo_model = TwoWindingModel(input=transformer)
trafo_model.calculate(is_sc=True)

# FEM calculation
trafo_model.fem_simulation(detailed_output = False)

Mesh Generator: Initial mesh generation
Mesh generator: Triangle
Mesh Generator: Reading initial mesh from memory
Solver: Solver - MUMPS (none)
Solver: Elapsed time: 00:01.307 s
Magnetic Energy 6.044139882628878
zb, ib: 0.25 ohm 909.33 A
SCI: 1.81 [%]
Bax  [HV] = 24.81 [mT]
Brad [HV] = 12.83 [mT]
Bax  [LV] = 9.72 [mT]
Brad [Lv] = 14.14 [mT]


The following command can list the main results of the calculation:

In [5]:
print(trafo_model.hv_winding)

WindingDesign(inner_radius=288.1, thickness=10.5, winding_height=354.83, filling_factor=7.37, current_density=25.53, mass=9.7, dc_loss=0.0, ac_loss=1566.1902253222818, outer_radius=298.6, cable_length=1192.47, amper_turns=7010.2)


In [6]:
print(trafo_model.results)

MainResults(wh=809.5, feasible=True, core_loss=2.4, load_loss=58109.17380108339, copper_mass=0.0, sci=1.75, window_width=139.2, core_mass=4868.1, turn_voltage=29.98, capitalized_cost=0.0, fem_based_sci=1.8084024240510435, fem_bax_hv=0.024813137840793836, fem_brad_hv=0.01283241597813145, fem_bax_brad_hv=0.0, fem_bax_lv=0.00971538840493106, fem_brad_lv=0.014136827733997492, fem_bax_brad_lv=0.0)


In [7]:
print(trafo_model.lv_winding)

WindingDesign(inner_radius=243.5, thickness=6.2, winding_height=342.5, filling_factor=14.69, current_density=22.46, mass=9.3, dc_loss=0.0, ac_loss=1501.5972080712397, outer_radius=249.7, cable_length=1143.29, amper_turns=7006.2)


In [8]:
trafo_model.fem_simulation(detailed_output = True)
print('analytical sci:', trafo_model.results.sci)

Mesh Generator: Initial mesh generation
Magnetic Energy 6.044139882661268
zb, ib: 0.25 ohm 909.33 A
SCI: 1.81 [%]
Mesh generator: Triangle
Mesh Generator: Reading initial mesh from memory
Solver: Solver - MUMPS (none)
Solver: Elapsed time: 00:01.067 s
Bax  [HV] = 24.81 [mT]
Brad [HV] = 12.83 [mT]
Bax  [LV] = 9.72 [mT]
Brad [Lv] = 14.14 [mT]
Values along the hv winding: [(0.011448245902903775, 0.01283241597813678), (0.019476743181043928, 0.005277299024413651), (0.021529942249883985, 0.0030880196074107857), (0.02273290836380036, 0.0020858069652822664), (0.023507255535646462, 0.0015538029128245867), (0.02401369686228544, 0.0012555736112749294), (0.02434391207424684, 0.0010897857405698375), (0.024562314093211288, 0.0010059437334852817), (0.02470028777097389, 0.000972378669308327), (0.024780066860041944, 0.0009718193434662403), (0.024813137840788084, 0.0010072843818638155), (0.024805456132087435, 0.0010452905829224646), (0.024755725928760547, 0.0010775025196329002), (0.02465620221949783, 0.

The resulting fem-based short circuit impedance is 3 %, which is differsthe given 2.45 %, however, the analytically calculated sci is only 1.7%. The reason is that the analytical formula contains the turn voltage, which is different, the core somehow not sized properly. 

In [9]:
from src.superconductor_losses import cryo_surface,cryostat_losses, magnusson_ac_loss, supra_winding_ac_loss
a_cs = cryo_surface(205.0, 370, 800)  # [mm2]
cryo_loss = cryostat_losses(a_cs)
#thermal_loss = thermal_incomes(self.input.required.lv.ph_current, self.input.required.hv.ph_current)

print('a_cs:',a_cs)
#print('cryostat losses:', cryo_loss)
print(cryo_loss)

# HV winding
print('hv:',magnusson_ac_loss(32.32*1e-3, 15.7*1e-3, 50, 34.5, Ic=115), 'W/m')
print('lv:',magnusson_ac_loss(21.9*1e-3, 17.9*1e-3, 50, 902/20, Ic=115), ',W/m')

# Calculation with the averaged values of Bax and Brad along the winding
B_hv = [(0.01525868673291042, 0.015692570721704158), (0.02493723180434261, 0.007042735634056996), (0.027783534999884618, 0.004255787627876373), (0.02943442875654105, 0.002901784324006673), (0.03049058541941392, 0.0021505210196979263), (0.031193411636048426, 0.0017174424410915101), (0.03165775253078697, 0.0014656848635876685), (0.03196390831189633, 0.001327468380940111), (0.0321614397283769, 0.0012595127807699669), (0.03227605832248241, 0.0012361306638610792), (0.032323261232306486, 0.0012719139638510124), (0.03231282332751821, 0.0013139105076056534), (0.03223679663461916, 0.001348350410761645), (0.03208732094592792, 0.0013635931732015605), (0.031858907620961205, 0.0013423325007524411), (0.03149036754115182, 0.0012643947440680314), (0.030934341149714092, 0.0010955798977385633), (0.030090638939423625, 0.0007692418202258633), (0.028782520690004337, 0.0008189194993197813), (0.026772412749399566, 0.0024756793245686026), (0.023501041803567213, 0.005883349029654816)]
print('hv average:',supra_winding_ac_loss(B_hv, 50, 69, Ic=170, kappa=2.0), 'W/m')

B_lv = [(0.010828182812545703, 0.01749291920202012), (0.015589700281180771, 0.006364106957295945), (0.017601199704695022, 0.0032317621554001485), (0.01901579745550995, 0.0016853521674473626), (0.02000551652061361, 0.0007729888722334266), (0.020692066220171543, 0.0001604041299998471), (0.021160221788117176, 0.00041294997330787453), (0.02147707382908226, 0.0007168691113257353), (0.02167963768959294, 0.0009766625584877175), (0.021799670677935645, 0.0012100289972609701), (0.02184985360091353, 0.0014576321985944052), (0.021839485962767843, 0.0016992919822627073), (0.02176594689856108, 0.0019515905811715584), (0.02162070425238405, 0.0022257452067311334), (0.021385663914935765, 0.0025463456121155303), (0.02102958618995092, 0.002939749888408727), (0.02050930935319842, 0.003456969227656259), (0.01976361371429998, 0.004200232860169351), (0.0187013344052194, 0.005410918360057693), (0.01716608027619833, 0.007768199125118613), (0.014065486259435198, 0.015587169161118908)]
print('lv average:',supra_winding_ac_loss(B_lv, 50, 69, Ic=170, kappa=1.5), 'W/m')


a_cs: 3188323.844311941
29.08
hv: 0.10514088717763595 W/m
lv: 0.1286420293250555 ,W/m
hv average: 0.031 W/m
lv average: 0.02 W/m


In [10]:
print("Cable length:")
r_m = 244
length = Np * 2*r_m*3.14 * 1e-3 
print(length*3)

length_sec = 10*22*2*3.14*1e-3*290
print(length_sec*3)


Cable length:
1204.40352
1201.9920000000002


In [11]:
P_ac_max = length*0.105*3 + length_sec*0.12*3
P_ac_avg = length*0.031*3 + length_sec*0.02*3

print('P_ac (max) = ', round(P_ac_max))
print('P_ac (avg) = ',round(P_ac_avg))

P_ac (max) =  271
P_ac (avg) =  61


In [12]:
pt = thermal_incomes(34.5,909)
print('Thermal losses:', pt, 'W')

Thermal losses: 254.75 W
